In [1]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [2]:
data_backup = data.iloc[3524:]
data = data.iloc[:3524]

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3524 entries, 0 to 3523
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    3524 non-null   object 
 1   open    3524 non-null   float64
 2   high    3524 non-null   float64
 3   low     3524 non-null   float64
 4   close   3524 non-null   float64
 5   volume  3524 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 165.3+ KB


### decision trees for 20 bar lag:

In [4]:
import pandas as pd

for i in range(1,21):
    data[f'close+{i}'] = data['close'].shift(i)
for i in range(1,21):
    data[f'high+{i}'] = data['high'].shift(i)
for i in range(1,21):
    data[f'low+{i}'] = data['low'].shift(i)
data.dropna(axis=0, inplace=True)

In [5]:
data.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'close+1', 'close+2',
       'close+3', 'close+4', 'close+5', 'close+6', 'close+7', 'close+8',
       'close+9', 'close+10', 'close+11', 'close+12', 'close+13', 'close+14',
       'close+15', 'close+16', 'close+17', 'close+18', 'close+19', 'close+20',
       'high+1', 'high+2', 'high+3', 'high+4', 'high+5', 'high+6', 'high+7',
       'high+8', 'high+9', 'high+10', 'high+11', 'high+12', 'high+13',
       'high+14', 'high+15', 'high+16', 'high+17', 'high+18', 'high+19',
       'high+20', 'low+1', 'low+2', 'low+3', 'low+4', 'low+5', 'low+6',
       'low+7', 'low+8', 'low+9', 'low+10', 'low+11', 'low+12', 'low+13',
       'low+14', 'low+15', 'low+16', 'low+17', 'low+18', 'low+19', 'low+20'],
      dtype='object')

In [6]:
inputs_close = data.iloc[:, [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]]
target_close = data.iloc[:, [4]]

inputs_high = data.iloc[:, [26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]]
target_high = data.iloc[:, [2]]

inputs_low = data.iloc[:, [46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65]]
target_low = data.iloc[:, [3]]

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(inputs_close, target_close, test_size=0.2, random_state=1, shuffle=False)
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(inputs_high, target_high, test_size=0.2, random_state=1, shuffle=False)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(inputs_low, target_low, test_size=0.2, random_state=1, shuffle=False)

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_c = MinMaxScaler()
scaler_c.fit(X_train_c)
scaler_target_c = MinMaxScaler()
scaler_target_c.fit(y_train_c)

scaler_h = MinMaxScaler()
scaler_h.fit(X_train_h)
scaler_target_h = MinMaxScaler()
scaler_target_h.fit(y_train_h)

scaler_l = MinMaxScaler()
scaler_l.fit(X_train_l)
scaler_target_l = MinMaxScaler()
scaler_target_l.fit(y_train_l)

X_train_c_scaled = scaler_c.transform(X_train_c)
X_test_c_scaled = scaler_c.transform(X_test_c)
y_train_c_scaled = scaler_target_c.transform(y_train_c)
y_test_c_scaled = scaler_target_c.transform(y_test_c)

X_train_h_scaled = scaler_h.transform(X_train_h)
X_test_h_scaled = scaler_h.transform(X_test_h)
y_train_h_scaled = scaler_target_h.transform(y_train_h)
y_test_h_scaled = scaler_target_h.transform(y_test_h)

X_train_l_scaled = scaler_l.transform(X_train_l)
X_test_l_scaled = scaler_l.transform(X_test_l)
y_train_l_scaled = scaler_target_l.transform(y_train_l)
y_test_l_scaled = scaler_target_l.transform(y_test_l)

random_tree_close = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_close.fit(X_train_c_scaled, y_train_c_scaled)

random_tree_high = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_high.fit(X_train_h_scaled, y_train_h_scaled)

random_tree_low = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_low.fit(X_train_l_scaled, y_train_l_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=1)

In [8]:
pred_close = random_tree_close.predict(X_test_c_scaled)
y_test_c['pred'] = pred_close

pred_high = random_tree_high.predict(X_test_h_scaled)
y_test_h['pred'] = pred_high

pred_low = random_tree_low.predict(X_test_l_scaled)
y_test_l['pred'] = pred_low

### data reading:

In [9]:
data_backup = pd.read_csv('data/^NDX_raw_data.csv')
data_backup.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [10]:
import pandas as pd

for i in range(1,21):
    data_backup[f'close+{i}'] = data_backup['close'].shift(i)
for i in range(1,21):
    data_backup[f'high+{i}'] = data_backup['high'].shift(i)
for i in range(1,21):
    data_backup[f'low+{i}'] = data_backup['low'].shift(i)
data_backup.dropna(axis=0, inplace=True)

In [11]:
inputs_close_backup = data_backup.iloc[:, [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]]
inputs_high_backup = data_backup.iloc[:, [26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]]
inputs_low_backup = data_backup.iloc[:, [46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65]]

In [28]:
df_pred = dict()

scaler_input = MinMaxScaler()
data_backup_c_scaled = scaler_input.fit_transform(inputs_close_backup)
scaler_target = MinMaxScaler()
target_backup_c_scaled = scaler_target.fit_transform(data_backup[['close']])

y_pred_c_backup = random_tree_close.predict(data_backup_c_scaled)
y_pred_c_backup = y_pred_c_backup.reshape(-1,1)
y_pred_c_backup = scaler_target.inverse_transform(y_pred_c_backup)

scaler_input = MinMaxScaler()
data_backup_h_scaled = scaler_input.fit_transform(inputs_high_backup)
scaler_target = MinMaxScaler()
target_backup_c_scaled = scaler_target.fit_transform(data_backup[['high']])

y_pred_h_backup = random_tree_high.predict(data_backup_h_scaled)
y_pred_h_backup = y_pred_h_backup.reshape(-1,1)
y_pred_h_backup = scaler_target.inverse_transform(y_pred_h_backup)

scaler_input = MinMaxScaler()
data_backup_l_scaled = scaler_input.fit_transform(inputs_low_backup)
scaler_target = MinMaxScaler()
target_backup_c_scaled = scaler_target.fit_transform(data_backup[['low']])

y_pred_l_backup = random_tree_low.predict(data_backup_l_scaled)
y_pred_l_backup = y_pred_l_backup.reshape(-1,1)
y_pred_l_backup = scaler_target.inverse_transform(y_pred_l_backup)

df_pred = {'pred_c':y_pred_c_backup, 'pred_h':y_pred_h_backup, 'pred_l':y_pred_l_backup}

In [29]:
data_backup.reset_index(inplace=True)

from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error

score_c = mean_absolute_error(data_backup[['close']], df_pred['pred_c'])
r2_c = r2_score(data_backup[['close']], df_pred['pred_c'])
data_backup['mae_c'] = score_c

score_h = mean_absolute_error(data_backup[['high']], df_pred['pred_h'])
r2_h = r2_score(data_backup[['high']], df_pred['pred_h'])
data_backup['mae_h'] = score_h

score_l = mean_absolute_error(data_backup[['low']], df_pred['pred_l'])
r2_l = r2_score(data_backup[['low']], df_pred['pred_l'])
data_backup['mae_l'] = score_l

print(f'METRICS FOR NASDAQ INDEX:')
print(f'score close: {score_c}')
print(f'score high: {score_h}')
print(f'score low: {score_l}')

print(f'- R2 METRICS FOR NASDAQ INDEX:')
print(f'r2 close: {r2_c}')
print(f'r2 high: {r2_h}')
print(f'r2 low: {r2_l}')

METRICS FOR NASDAQ INDEX:
score close: 94.5584431899241
score high: 76.04240560678421
score low: 88.01970598916931
- R2 METRICS FOR NASDAQ INDEX:
r2 close: 0.9989852858789718
r2 high: 0.9992946272841878
r2 low: 0.9991219265079113


In [37]:
remaining_columns = ('date','open','high','low','close','volume','mae_c','mae_h','mae_l')

data_backup = data_backup[list(remaining_columns)]

y_pred_c_series = pd.Series(df_pred['pred_c'].reshape(1,-1)[0])
y_pred_h_series = pd.Series(df_pred['pred_h'].reshape(1,-1)[0])
y_pred_l_series = pd.Series(df_pred['pred_l'].reshape(1,-1)[0])
data_backup['pred_close'] = y_pred_c_series
data_backup['pred_high'] = y_pred_h_series
data_backup['pred_low'] = y_pred_l_series

data_backup.to_excel(f'data_w_pred_m3/index_w_pred_m3.xlsx')